In [4]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_data = torchvision.datasets.CIFAR10(root="data/",download=True, train=True, transform=transform)

Files already downloaded and verified


In [37]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [38]:
class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=64,kernel_size=3,stride=1,padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,stride=1,padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3,stride=1,padding=1)
        self.pool = torch.nn.MaxPool2d(2,stride=2)
        self.fc1 = torch.nn.Linear(256 * 4 * 4, 128)  # Fully connected layer
        self.fc2 = torch.nn.Linear(128, 10)  # 10 output classes for CIFAR-10
        return
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x,start_dim=1,end_dim=-1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [39]:
a = iter(trainloader)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [41]:
model = CNN_Model().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [42]:
for epoch in range(20):  # Training for 10 epochs
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

Epoch 1, Loss: 1.2852
Epoch 2, Loss: 0.8513
Epoch 3, Loss: 0.6710
Epoch 4, Loss: 0.5429
Epoch 5, Loss: 0.4381
Epoch 6, Loss: 0.3472
Epoch 7, Loss: 0.2743
Epoch 8, Loss: 0.2255
Epoch 9, Loss: 0.1817
Epoch 10, Loss: 0.1571
Epoch 11, Loss: 0.1380
Epoch 12, Loss: 0.1281
Epoch 13, Loss: 0.1213
Epoch 14, Loss: 0.1060
Epoch 15, Loss: 0.1092
Epoch 16, Loss: 0.1015
Epoch 17, Loss: 0.0954
Epoch 18, Loss: 0.0945
Epoch 19, Loss: 0.0879
Epoch 20, Loss: 0.0834


In [43]:
test_data = torchvision.datasets.CIFAR10(root="data/",download=True, train=False, transform=transform)

Files already downloaded and verified


In [44]:
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [45]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 74.01%
